<a href="https://colab.research.google.com/github/siddharthkalyavit2025/ECOVISION-Trained-cnn-model/blob/main/Ecovision_new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow scikit-learn matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os, shutil, random

SOURCE_DIR = "/content/drive/MyDrive/Final_Waste_dataset"
DEST_DIR   = "/content/drive/MyDrive/waste_split"

TRAIN_RATIO, VAL_RATIO = 0.7, 0.15
random.seed(42)

classes = [c for c in os.listdir(SOURCE_DIR)
           if os.path.isdir(os.path.join(SOURCE_DIR, c))]

for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(DEST_DIR, split, cls), exist_ok=True)

for cls in classes:
    imgs = [i for i in os.listdir(os.path.join(SOURCE_DIR, cls))
            if i.lower().endswith(('.jpg','.jpeg','.png'))]
    random.shuffle(imgs)

    n = len(imgs)
    t_end = int(n * TRAIN_RATIO)
    v_end = t_end + int(n * VAL_RATIO)

    for i in imgs[:t_end]:
        shutil.copy(os.path.join(SOURCE_DIR, cls, i),
                    os.path.join(DEST_DIR, "train", cls, i))
    for i in imgs[t_end:v_end]:
        shutil.copy(os.path.join(SOURCE_DIR, cls, i),
                    os.path.join(DEST_DIR, "val", cls, i))
    for i in imgs[v_end:]:
        shutil.copy(os.path.join(SOURCE_DIR, cls, i),
                    os.path.join(DEST_DIR, "test", cls, i))


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.25,
    brightness_range=[0.7,1.3],
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/drive/MyDrive/waste_split/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_gen.flow_from_directory(
    "/content/drive/MyDrive/waste_split/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_gen.flow_from_directory(
    "/content/drive/MyDrive/waste_split/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 12171 images belonging to 6 classes.
Found 2607 images belonging to 6 classes.
Found 2612 images belonging to 6 classes.


In [5]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = train_data.classes
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

class_weights = dict(enumerate(class_weights))
print(class_weights)


{0: np.float64(2.065682281059063), 1: np.float64(1.4489285714285713), 2: np.float64(1.2024303497332542), 3: np.float64(1.1176308539944904), 4: np.float64(0.48447575829949846), 5: np.float64(0.965952380952381)}


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False  # first stage

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(base_model.input, output)


In [7]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_waste_model.h5", save_best_only=True)
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    class_weight=class_weights,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 616ms/step - accuracy: 0.6355 - loss: 0.9080

381/381 ━━━━━━━━━━━━━━━━━━━━ 294s 698ms/step - accuracy: 0.6357 - loss: 0.9076 - val_accuracy: 0.8067 - val_loss: 0.5642
Epoch 2/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.7872 - loss: 0.5274

381/381 ━━━━━━━━━━━━━━━━━━━━ 239s 626ms/step - accuracy: 0.7872 - loss: 0.5274 - val_accuracy: 0.8278 - val_loss: 0.4877
Epoch 3/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 582ms/step - accuracy: 0.8199 - loss: 0.4364

381/381 ━━━━━━━━━━━━━━━━━━━━ 237s 622ms/step - accuracy: 0.8199 - loss: 0.4364 - val_accuracy: 0.8400 - val_loss: 0.4548
Epoch 4/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.8448 - loss: 0.3809

381/381 ━━━━━━━━━━━━━━━━━━━━ 239s 626ms/step - accuracy: 0.8448 - loss: 0.3809 - val_accuracy: 0.8462 - val_loss: 0.4389
Epoch 5/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 232s 609ms/step - accuracy: 0.8625 - loss: 0.3329 - val_accuracy: 0.8485 - val_loss: 0.4476
Epoch 6/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 572ms/step - accuracy: 0.8741 - loss: 0.3015

381/381 ━━━━━━━━━━━━━━━━━━━━ 232s 610ms/step - accuracy: 0.8740 - loss: 0.3016 - val_accuracy: 0.8596 - val_loss: 0.4202
Epoch 7/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 562ms/step - accuracy: 0.8806 - loss: 0.2895

381/381 ━━━━━━━━━━━━━━━━━━━━ 228s 599ms/step - accuracy: 0.8806 - loss: 0.2895 - val_accuracy: 0.8569 - val_loss: 0.4170
Epoch 8/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.8974 - loss: 0.2506

381/381 ━━━━━━━━━━━━━━━━━━━━ 231s 606ms/step - accuracy: 0.8974 - loss: 0.2506 - val_accuracy: 0.8734 - val_loss: 0.3758
Epoch 9/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 230s 603ms/step - accuracy: 0.9035 - loss: 0.2323 - val_accuracy: 0.8646 - val_loss: 0.4074
Epoch 10/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 229s 601ms/step - accuracy: 0.9095 - loss: 0.2191 - val_accuracy: 0.8753 - val_loss: 0.4034
Epoch 11/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 563ms/step - accuracy: 0.9106 - loss: 0.2101

381/381 ━━━━━━━━━━━━━━━━━━━━ 229s 601ms/step - accuracy: 0.9106 - loss: 0.2101 - val_accuracy: 0.8780 - val_loss: 0.3649
Epoch 12/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 228s 600ms/step - accuracy: 0.9198 - loss: 0.1929 - val_accuracy: 0.8750 - val_loss: 0.3809
Epoch 13/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 228s 599ms/step - accuracy: 0.9263 - loss: 0.1666 - val_accuracy: 0.8696 - val_loss: 0.4146
Epoch 14/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 228s 598ms/step - accuracy: 0.9244 - loss: 0.1742 - val_accuracy: 0.8845 - val_loss: 0.3733
Epoch 15/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 229s 601ms/step - accuracy: 0.9276 - loss: 0.1664 - val_accuracy: 0.8788 - val_loss: 0.3834
Epoch 16/40
381/381 ━━━━━━━━━━━━━━━━━━━━ 228s 597ms/step - accuracy: 0.9316 - loss: 0.1604 - val_accuracy: 0.8707 - val_loss: 0.4124


In [9]:
test_loss, test_acc = model.evaluate(test_data)
print("Test Accuracy:", test_acc)


82/82 ━━━━━━━━━━━━━━━━━━━━ 25s 304ms/step - accuracy: 0.8791 - loss: 0.3806
Test Accuracy: 0.8767228126525879


In [10]:
model.save("waste_classifier_model.keras")
